In [1]:
import numpy as np
import pandas as pd
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt 
from keras.regularizers import l2
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential

C:\Users\Naoki Tomita\Anaconda3\envs\kaggle\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('../input/train_1.csv')
test = pd.read_csv('../input/test_1.csv')
target = train['target']
del train['target']

In [4]:
y = target

train_columns = [c for c in train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
train_df = train[train_columns]
test_df = test[train_columns]

#####Handling Missing Values#####     

for i in range(len(train_df.columns)):
    train_df.iloc[:,i] = (train_df.iloc[:,i]).fillna(-1)

for i in range(len(test_df.columns)):
    test_df.iloc[:,i] = (test_df.iloc[:,i]).fillna(-1)    
    
####Normalizing the values####

mmScale = MinMaxScaler()

n = train_df.shape[1]
print('n is:',n)

x_train = mmScale.fit_transform(train_df)
x_test = mmScale.transform(test_df)


from keras import optimizers
from keras import layers
from keras.layers import LeakyReLU

#####################Early Stopping ########################
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
#####Artificial Neural Networks Implementation#####
print("Starting Neural Network")

model_n = Sequential()
#Want to use an expotential linear unit instead of the usual relu
#model_n.add( Dense( n, activation='relu', input_shape=(n,) ) )
model_n.add( Dense( n, activation='tanh', input_shape=(n,) ) )
model_n.add(BatchNormalization())
model_n.add( Dense( int(0.5*n), activation='relu' ) )
model_n.add(LeakyReLU(alpha=.0011))

model_n.add(Dropout(0.5))

model_n.add(Dense(1, activation='linear'))
model_n.compile(loss='mse', optimizer='Adadelta',  metrics=['mse'])
        

model_n.fit(x_train, y, epochs=25,verbose=1, batch_size=32,validation_split=0.2, callbacks=[early_stopping])

predictions = model_n.predict(x_test)

C:\Users\Naoki Tomita\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


n is: 222
Starting Neural Network
Train on 161533 samples, validate on 40384 samples
Epoch 1/25
161533/161533 [==============================] - 20s 122us/step - loss: 14.2814 - mean_squared_error: 14.2814 - val_loss: 14.5480 - val_mean_squared_error: 14.5480
Epoch 2/25
161533/161533 [==============================] - 17s 103us/step - loss: 14.0332 - mean_squared_error: 14.0332 - val_loss: 14.3969 - val_mean_squared_error: 14.3969
Epoch 3/25
161533/161533 [==============================] - 16s 101us/step - loss: 13.9761 - mean_squared_error: 13.9761 - val_loss: 14.4797 - val_mean_squared_error: 14.4797
Epoch 4/25
161533/161533 [==============================] - 17s 104us/step - loss: 13.9363 - mean_squared_error: 13.9363 - val_loss: 14.5792 - val_mean_squared_error: 14.5792
Epoch 5/25
161533/161533 [==============================] - 17s 103us/step - loss: 13.9155 - mean_squared_error: 13.9155 - val_loss: 14.3455 - val_mean_squared_error: 14.3455
Epoch 6/25
161533/161533 [==============

In [5]:
sub_df = pd.read_csv("../input//sample_submission.csv")
sub_df["target"] = predictions
sub_df.to_csv("submit_keras.csv", index=False)

# SCORE 3.805